# Assign genus IDs based on pyANI analysis, and piecewise linear regression boundaries, and other.

**Set Up**

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import networkx as nx
import dwave_networkx as dnx
import ete3
from ete3 import Tree, TreeStyle, faces
from ete3 import PhyloNode
from ete3 import NodeStyle
from collections import defaultdict

**Loading data**
Here we are interested in coverage and identity matrices from pyANI analysis.

In [2]:
identity = pd.read_csv(Path("../../supplementary_file_3/output/pyani_matrices/matrix_identity_1.tab"), sep='\t').rename(columns={'Unnamed: 0': 'genome1'})
coverage = pd.read_csv(Path("../../supplementary_file_3/output/pyani_matrices/matrix_coverage_1.tab"), sep='\t').rename(columns={'Unnamed: 0': 'genome1'})

In [3]:
pd.read_csv(Path("../../supplementary_file_3/output/pyani_matrices/matrix_identity_1.tab"), sep='\t').rename(columns={'Unnamed: 0': ''})

,,GCF_000009765.2 | Streptomyces avermitilis MA-4680 - ST 160:1,GCF_000010605.1 | Streptomyces griseus NBRC 13350 - ST 12:2,GCF_000092385.1 | Streptomyces bingchenggensis BCW-1 - ST 238:3,GCF_000147815.2 | Streptomyces violaceusniger Tu 4113 - ST 239:4,GCF_000225525.1 | Streptomyces zinciresistens K42 - ST 236:5,GCF_000226455.1 | Streptomyces chartreusis NRRL 12338 - ST 242:6,GCF_000261345.2 | Streptomyces globisporus C-1027 - ST 245:7,GCF_000297635.1 | Streptomyces sp. AA0539 - ST 246:8,GCF_000317595.1 | Streptomyces ipomoeae 91-03 - ST 248:9,...,GCF_900095575.1 | Streptomyces sp. DpondAA-F4 - ST 241:286,GCF_900104815.1 | Streptomyces misionensis DSM 40306 - ST 795:287,GCF_900105245.1 | Streptomyces sp. 1222.5 - ST 474:288,GCF_900105265.1 | Streptomyces sp. 3213 - ST 796:289,GCF_900105395.1 | Streptomyces sp. TLI_053 - ST 797:290,GCF_900119365.1 | Streptomyces atratus OK807 - ST 801:291,GCF_900187385.1 | Streptomyces sp. 2114.4 - ST 466:292,GCF_900206255.1 | Streptomyces sp. TLI_55 - ST 803:293,GCF_900215595.1 | Streptomyces sp. 1222.2 - ST 804:294,GCF_900215615.1 | Streptomyces sp. Ag82_G6-1 - ST 805:295
0,GCF_000009765.2 | Streptomyces avermitilis MA-...,1.000000,0.852647,0.848301,0.846451,0.860136,0.861142,0.852745,0.840168,0.862197,...,0.852510,0.858158,0.859093,0.861743,0.841550,0.853517,0.851268,0.860556,0.861914,0.859906
1,GCF_000010605.1 | Streptomyces griseus NBRC 13...,0.852647,1.000000,0.848414,0.849499,0.855114,0.850535,0.913605,0.842829,0.851632,...,0.866897,0.852236,0.850801,0.850989,0.843457,0.868380,0.854658,0.850430,0.851067,0.850803
2,GCF_000092385.1 | Streptomyces bingchenggensis...,0.848301,0.848414,1.000000,0.868619,0.847408,0.850079,0.847756,0.845074,0.848350,...,0.847729,0.849378,0.847759,0.844698,0.842863,0.849720,0.853129,0.846544,0.847519,0.846097
3,GCF_000147815.2 | Streptomyces violaceusniger ...,0.846451,0.849499,0.868619,1.000000,0.849283,0.848356,0.847364,0.845778,0.848294,...,0.848101,0.848276,0.846214,0.845147,0.842030,0.849844,0.853668,0.844952,0.846911,0.845076
4,GCF_000225525.1 | Streptomyces zinciresistens ...,0.860136,0.855114,0.847408,0.849283,1.000000,0.865323,0.850819,0.843039,0.861825,...,0.850570,0.863009,0.863090,0.859476,0.843049,0.851173,0.850074,0.865115,0.862412,0.864954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,GCF_900119365.1 | Streptomyces atratus OK807 -...,0.853517,0.868380,0.849720,0.849844,0.851173,0.851486,0.867223,0.843128,0.851463,...,0.868157,0.851958,0.850235,0.850825,0.842531,1.000000,0.854370,0.851004,0.852262,0.851125
291,GCF_900187385.1 | Streptomyces sp. 2114.4 - ST...,0.851268,0.854658,0.853129,0.853668,0.850074,0.849862,0.854988,0.846363,0.850021,...,0.853113,0.851489,0.850524,0.848857,0.844373,0.854370,1.000000,0.849781,0.849851,0.848978
292,GCF_900206255.1 | Streptomyces sp. TLI_55 - ST...,0.860556,0.850430,0.846544,0.844952,0.865115,0.865422,0.851598,0.840800,0.862855,...,0.849136,0.861922,0.862419,0.866312,0.841969,0.851004,0.849781,1.000000,0.863221,0.865114
293,GCF_900215595.1 | Streptomyces sp. 1222.2 - ST...,0.861914,0.851067,0.847519,0.846911,0.862412,0.862262,0.851766,0.840843,0.882600,...,0.851211,0.859853,0.860631,0.860754,0.841926,0.852262,0.849851,0.863221,1.000000,0.861905


**Assigning species and genus ID**
Here, we:
- write a function that removes suffix provided in the pyANI matrices
- get a function that will use provided dataframe to calculate MST, and remove edges between genomes if their thereshold is lower than the one provided. In case, where cliques are present it will keep removing the lowest weight until no cliques are present. 

In [4]:
def remove_suffix(row, col):
    """Return accession number from 'FILE' column."""

    try:
        new = ' '.join(row[col].split(':')[:-1])
    except IndexError:
        new = 'NA'

    return new



In [5]:
def assign_ANI_tax_ID_one_attribute(df, thereshold, attribute, ANI_ID):
        """Assign taxon IDs based on ANI analysis. 
        
        :param df: dataframe with each row providing pyANI identity and coverage for a given parir of genomes
        :param thereshold: thereshold at which genomes should be separated (num)
        :param attribute: comparision type deciding which should be used to separate genomes identity or coverage
        :param ANI_ID: current number of assigned IDs
        """
        
        #Generate NetworkX grah with identity and coverage as edge attibutes 
        G_comp=nx.from_pandas_edgelist(df, 'genome1', 'genome2', ['identity', 'coverage'])
        
        current_assignments = {} #Hold empty dictionary

        ANI_ID = ANI_ID

        #Remove edges if thereshold for a given attribute is lower 
        edges_to_remove = [(n1,n2) for n1, n2, attrs in G_comp.edges(data=True) if attrs[attribute] < thereshold]
        G_comp.remove_edges_from(edges_to_remove)
        
        #Check if the components are clique, if not remove edges until clique is achived
        components = [_ for _ in list(nx.connected_components(G_comp))]
        for component in components:
            while dnx.is_clique(G_comp, component) == False:
                weights = sorted(list(set([attrs[attribute] for n1, n2, attrs in G_comp.edges(data=True) if n1 in component and n2 in component])))
                edges_to_remove = [(n1,n2) for n1, n2, attrs in G_comp.edges(data=True) if attrs[attribute] < (weights)[1] and n1 in component and n2 in component]
                G_comp.remove_edges_from(edges_to_remove)
                weights.remove(weights[0])

                components = [_ for _ in list(nx.connected_components(G_comp))]
                for component in components:
                    if dnx.is_clique(G_comp, component) == True:
                        break
        #Assign IDs
        components = [_ for _ in list(nx.connected_components(G_comp))]
        for component in components:
            current_assignments.update({_:ANI_ID for _ in component})
            ANI_ID += 1

    
        return current_assignments

In [6]:
current_species_ID = 1
current_genus_ID = 1
genome_genus_ID = {}
genome_species_ID = {}
                   #Melting data
identity_melt = pd.melt(identity, id_vars=['genome1'], value_vars=[_ for _ in identity if _ != 'genome1'], var_name='genome2', value_name='identity')
coverage_melt = pd.melt(coverage, id_vars=['genome1'], value_vars=[_ for _ in identity if _ != 'genome1'], var_name='genome2', value_name='coverage')
    
                    #Combine data
combined = pd.merge(identity_melt, coverage_melt,  how='left', left_on=['genome1', 'genome2'], right_on = ['genome1','genome2'])
combined = combined[combined['genome1'] != combined['genome2']] #Remove self-to-self comparisions
                    #Remove duplicate comparisions and keep minimum coverage and average identity
combined[['genome1','genome2']] = np.sort(combined[['genome1','genome2']].to_numpy(),axis=1)
fixed = (combined.groupby(['genome1','genome2']).agg(identity = ('identity','mean'), coverage = ('coverage','min')).reset_index())
fixed["genome1"] = fixed.apply(remove_suffix,col='genome1', axis=1)
fixed['genome2'] = fixed.apply(remove_suffix,col='genome2', axis=1)


**Generate dataframe to which we will append information**

In [7]:
df = pd.read_csv(Path("../../supplementary_file_3/input/custom_labels.txt"), sep='\t', names=['MD5_hash', 'FILE', 'label'])

In [8]:
def get_accession(row):
    """Return accession number from 'FILE' column."""

    try:
        acc = '_'.join(row['FILE'].split('_')[:2])
    except IndexError:
        acc = 'NA'

    return acc

df["accession"] = df.apply(get_accession, axis=1)

In [9]:
del df['MD5_hash']
del df['FILE']

**Assigning genus IDs using boundries with 4 segment piecewise regression**

Here, we only consider one attribute; genome coverage. 

In [10]:
#Getting Genus ID
groups = assign_ANI_tax_ID_one_attribute(fixed, 0.556, 'coverage', current_genus_ID)
labels_to_accession = df.set_index('label').to_dict()['accession']
genome_genus_ID = {labels_to_accession[label]:genus_ID for label,genus_ID in groups.items()}

In [11]:
df['genus_ID_pc_4'] = df['accession'].map(genome_genus_ID)

In [12]:
df

,label,accession,genus_ID_pc_4
0,GCF_000009765.2 | Streptomyces avermitilis MA-...,GCF_000009765.2,1
1,GCF_000010605.1 | Streptomyces griseus NBRC 13...,GCF_000010605.1,2
2,GCF_000092385.1 | Streptomyces bingchenggensis...,GCF_000092385.1,3
3,GCF_000147815.2 | Streptomyces violaceusniger ...,GCF_000147815.2,4
4,GCF_000225525.1 | Streptomyces zinciresistens ...,GCF_000225525.1,5
...,...,...,...
290,GCF_900119365.1 | Streptomyces atratus OK807 -...,GCF_900119365.1,72
291,GCF_900187385.1 | Streptomyces sp. 2114.4 - ST...,GCF_900187385.1,33
292,GCF_900206255.1 | Streptomyces sp. TLI_55 - ST...,GCF_900206255.1,61
293,GCF_900215595.1 | Streptomyces sp. 1222.2 - ST...,GCF_900215595.1,14


**Assigining genus/species IDs based on two attributes.**

Here, we will write a function that will assign genomes to candidate genus and/or species by considering both genome coverage and genome identity. 

The function will work in the following steps:
- Create a complete graph, where each genome/node is connected to evry other genome/node
- Assign genome coverage and genome identity as edges attributes
- Remove edges if the thereshold for genome covera is lower than the one provodes. In case, where cliques are present it will keep removing the lowest weigh until no cliques are present. 
- Then, we will do the same but this time with genome identity for a given/specified genome identity threshold. 



In [13]:
def assign_ANI_tax_ID_two_attribute(df, threshold_1, attribute_1, ANI_ID, attribute_2, threshold_2):
        """Assign taxon IDs based on ANI analysis. 
        
        :param df: dataframe with each row providing pyANI identity and coverage for a given pair of genomes
        :param thereshold: thereshold at which genomes should be separated (float)
        :param attribute: comparision type deciding which should be used to separate genomes (identity or coverage)
        :param ANI_ID: current number of assigned IDs
        """
        
        #Generate NetworkX grah with identity and coverage as edge attibutes 
        G_comp=nx.from_pandas_edgelist(df, 'genome1', 'genome2', ['identity', 'coverage'])
        
        
        
        current_assignments = {} #Hold empty dictionary

        ANI_ID = ANI_ID

        
        
        edges_to_remove = [(n1,n2) for n1, n2, attrs in G_comp.edges(data=True) if attrs[attribute_1] < threshold_1 or attrs[attribute_2] < threshold_2]
        G_comp.remove_edges_from(edges_to_remove)
        print(G_comp)
        nx.write_graphml_lxml(G_comp, Path("graph.graphml"))
        
        
        #Check if the components are clique, if not remove edges until clique is achived
        components = [_ for _ in list(nx.connected_components(G_comp))]
        for component in components:
            while dnx.is_clique(G_comp, component) == False:
                print(len(component))
                weights = sorted(list(set([attrs[attribute_1] for n1, n2, attrs in G_comp.edges(data=True) if n1 in component and n2 in component])))
                edges_to_remove = [(n1,n2) for n1, n2, attrs in G_comp.edges(data=True) if attrs[attribute_1] < (weights)[1] and n1 in component and n2 in component]
                G_comp.remove_edges_from(edges_to_remove)
                weights.remove(weights[0])

                components = [_ for _ in list(nx.connected_components(G_comp))]
                for component in components:
                    if dnx.is_clique(G_comp, component) == True:
                        break
        #Assign IDs
        components = [_ for _ in list(nx.connected_components(G_comp))]
        for component in components:
            current_assignments.update({_:ANI_ID for _ in component})
            ANI_ID += 1

        nx.write_graphml_lxml(G_comp, Path("graph2.graphml"))
    
        return current_assignments

In [14]:
groups = assign_ANI_tax_ID_two_attribute(fixed, 0.556, 'coverage', current_genus_ID, 'identity', 0.881)
labels_to_accession = df.set_index('label').to_dict()['accession']
genome_genus_ID = {labels_to_accession[label]:genus_ID for label,genus_ID in groups.items()}

Graph with 295 nodes and 1054 edges
19
8
6
16
16
20
15
14
10
17
7
4


In [15]:
df['genus_ID_pc_4_with_ID'] = df['accession'].map(genome_genus_ID)

In [16]:
df

,label,accession,genus_ID_pc_4,genus_ID_pc_4_with_ID
0,GCF_000009765.2 | Streptomyces avermitilis MA-...,GCF_000009765.2,1,1
1,GCF_000010605.1 | Streptomyces griseus NBRC 13...,GCF_000010605.1,2,2
2,GCF_000092385.1 | Streptomyces bingchenggensis...,GCF_000092385.1,3,3
3,GCF_000147815.2 | Streptomyces violaceusniger ...,GCF_000147815.2,4,4
4,GCF_000225525.1 | Streptomyces zinciresistens ...,GCF_000225525.1,5,5
...,...,...,...,...
290,GCF_900119365.1 | Streptomyces atratus OK807 -...,GCF_900119365.1,72,72
291,GCF_900187385.1 | Streptomyces sp. 2114.4 - ST...,GCF_900187385.1,33,33
292,GCF_900206255.1 | Streptomyces sp. TLI_55 - ST...,GCF_900206255.1,61,61
293,GCF_900215595.1 | Streptomyces sp. 1222.2 - ST...,GCF_900215595.1,14,14


For piecewise with 2 segements

**Checking if the groups are monophyletic in SCO tree.**

One of the main objectives of carrying this anlysis was to identify biologically meaningful groups for pangenomic analysis. 

After mapping the identified genus for genome coverage theresholds identified by piecewise regression, it was found that the groups do not form monophyletic groups on SCO tree. 

Therefore, to find groupings in which that problem does not occur the analysis will be run at starting genome coverage between 40% and 60% in steps of 0.1%.

**Check monophyly**

Writing function that will check if a given set of genomes form monophyletic caldes in the SCO phylogenetic tree.

In [21]:
def check_monophyly(tree, group):
    """Return True if teh given group
    is monophyletic, otherwise return False.
    """
    
    monophyly_status = tree.check_monophyly(values=group, target_attr="name", ignore_missing=True)[0]
    
    
    return monophyly_status

In [17]:
SCO_tree = Tree('../../supplementary_file_5/output/tree/04_tbe.raxml.support', format=1)

R = SCO_tree.get_midpoint_outgroup()
# and set it as tree outgroup
SCO_tree.set_outgroup(R)

**Assigning genus IDs with starting genome coverage between 45% and 55% in steps of 0.1%**

In [24]:
genus_df = pd.DataFrame(columns=['coverage_threshold', 'clusters', 'singletons', 'monophyletic', 'non_monophyletic'])

for i in np.arange(0.400, 0.851, 0.001):
    
    cluster_members = defaultdict(list) #Hold an empty defaultdict; here members of the same group will be keyed by asigned genus ID
    groups = assign_ANI_tax_ID_one_attribute(fixed, round(i, 3), 'coverage', current_genus_ID) #Assigning genus IDs
    labels_to_accession = df.set_index('label').to_dict()['accession']
    genome_genus_ID = {labels_to_accession[label]:genus_ID for label,genus_ID in groups.items()} #Mapping genome accessions
    #Get cluster/genus ID members
    for genome, genus_ID in genome_genus_ID.items():
        cluster_members[genus_ID].append(genome)
    
    no_of_monophyletic_groups = 0
    no_of_non_monophyletic_groups = 0
    singleton = 0
    #Check monophyly of all groups with at least 2 genomes
    for k, v in cluster_members.items():
        if len(v) !=1:
            monophyly_stat = check_monophyly(SCO_tree, v)
            if monophyly_stat == True:
                no_of_monophyletic_groups +=1
            else:
                no_of_non_monophyletic_groups += 1
        else:
            singleton += 1
    genus_df.loc[len(genus_df)] = [round(i*100, 3), len(cluster_members), singleton ,int(no_of_monophyletic_groups), int(no_of_non_monophyletic_groups)]
    

In [20]:
genus_df[['clusters', 'singletons', 'monophyletic', 'non_monophyletic']] = genus_df[['clusters', 'singletons', 'monophyletic', 'non_monophyletic']].applymap(np.int64)

In [21]:
genus_df.to_csv(Path("../output/monophyly_status.csv").expanduser(), index=False)

**Reorder df**

Here, we will reorder dataframe so that the accessions match the ordering of the leave nodes in the SCO tree. 

In [18]:
data2 = pd.read_csv(Path("../../supplementary_file_5/output/SCOG_tree_node_order.csv").expanduser())

In [19]:
data2 = pd.merge(data2, df, on='accession')

In [20]:
del data2['label']

In [21]:
data2

,accession,genus_ID_pc_4,genus_ID_pc_4_with_ID
0,GCF_000709915.1,20,20
1,GCF_018966745.1,20,20
2,GCF_900079415.1,20,20
3,GCF_000719785.1,20,20
4,GCF_002899455.1,20,20
...,...,...,...
290,GCF_000813365.1,40,40
291,GCF_000745345.1,38,38
292,GCF_900105395.1,79,79
293,GCF_000717725.1,24,24


In [22]:
data2.to_csv(Path("../output/pyANI_genus_IDs.csv").expanduser(), index=False)